In [1]:
import pandas as pd
import numpy as np

In [2]:
#read data
calories3 = pd.read_json('../osfstorage-archive/pmdata/p03/fitbit/calories.json')
distance3 = pd.read_json('../osfstorage-archive/pmdata/p03/fitbit/distance.json')
l_act3 = pd.read_json('../osfstorage-archive/pmdata/p03/fitbit/lightly_active_minutes.json')
m_act3 = pd.read_json('../osfstorage-archive/pmdata/p03/fitbit/moderately_active_minutes.json')
r_heart3 = pd.read_json('../osfstorage-archive/pmdata/p03/fitbit/resting_heart_rate.json')
sedentary3 = pd.read_json('../osfstorage-archive/pmdata/p03/fitbit/sedentary_minutes.json')
sleep_score3 = pd.read_csv('../osfstorage-archive/pmdata/p03/fitbit/sleep_score.csv')
sleep3 = pd.read_json('../osfstorage-archive/pmdata/p03/fitbit/sleep.json')
steps3 = pd.read_json('../osfstorage-archive/pmdata/p03/fitbit/steps.json')
hr_zone3 = pd.read_json('../osfstorage-archive/pmdata/p03/fitbit/time_in_heart_rate_zones.json')
v_act3 = pd.read_json('../osfstorage-archive/pmdata/p03//fitbit/very_active_minutes.json')

In [3]:
#data frequency to date using sum
calories_processed = calories3.resample('D', on = 'dateTime').value.sum()
distance_processed = distance3.resample('D', on = 'dateTime').value.sum()
steps_processed = steps3.resample('D', on = 'dateTime').value.sum()

In [4]:
#resting heart rate to simple df
r_heart3['value']=r_heart3['value'].apply(lambda x: x['value'])
r_heart_processed=r_heart3

In [5]:
#delete time in sleep score time stamp
sleep_score3['timestamp']=sleep_score3['timestamp'].apply(lambda x: x[:10])
#drop unnecessary column
sleep_score_processed = sleep_score3.drop(columns='sleep_log_entry_id') #drop entry id

In [6]:
#dictionary with heart rate zone minutes to df
hr_zone_value = pd.DataFrame([eval(str(a).replace('{\'valuesInZones\': ','').replace('}}','}')) for a in hr_zone3['value']])
hr_zone_processed = pd.concat([hr_zone3['dateTime'], hr_zone_value], axis=1)

In [7]:
#unify index
l_act_processed = l_act3.set_index('dateTime')
m_act_processed = m_act3.set_index('dateTime')
r_heart_processed = r_heart_processed.set_index('dateTime')
sedentary_processed = sedentary3.set_index('dateTime')
sleep_score_processed = sleep_score_processed.set_index('timestamp')
sleep_processed = sleep3.set_index('dateOfSleep')
hr_zone_processed = hr_zone_processed.set_index('dateTime')
v_act_processed = v_act3.set_index('dateTime')
#index to DateTimeIndex
sleep_score_processed.index = pd.to_datetime(sleep_score_processed.index)

In [8]:
#get dummy values for mainsleep true(1)/false(0)
sleep_processed['mainSleep'] = pd.get_dummies(sleep_processed['mainSleep'])[True].fillna(0)
#drop unnecessary columns
sleep_processed = sleep_processed.drop(['logId','startTime','endTime','type','infoCode','levels'], axis=1)
#index to DateTimeIndex
sleep_processed.index = pd.to_datetime(sleep_processed.index)

In [9]:
#read prediction target
wellness = pd.read_csv('../osfstorage-archive/pmdata/p03/pmsys/wellness.csv') 
wellness['effective_time_frame'] = wellness['effective_time_frame'].apply(lambda x: x[:10])
wellness = wellness.rename(columns={"effective_time_frame": "dateTime"})
readiness = wellness.loc[:,['dateTime','readiness']].set_index('dateTime')
readiness.index = pd.to_datetime(readiness.index)

In [10]:
#drop duplicated records
readiness, calories_processed,distance_processed,l_act_processed,m_act_processed,r_heart_processed,sedentary_processed,sleep_score_processed,sleep_processed,steps_processed,hr_zone_processed,v_act_processed = [x[~x.index.duplicated(keep='first')] for x in [readiness, calories_processed,distance_processed,l_act_processed,m_act_processed,r_heart_processed,sedentary_processed,sleep_score_processed,sleep_processed,steps_processed,hr_zone_processed,v_act_processed]]

In [11]:
#concat data
P03 = pd.concat([readiness, calories_processed,distance_processed,l_act_processed,m_act_processed,r_heart_processed,sedentary_processed,sleep_score_processed,sleep_processed,steps_processed,hr_zone_processed,v_act_processed], axis=1)

In [12]:
#rename columns
P03.columns=['readiness','calories','distance','l_act','m_act','resting_heart','sedentary','sleep_overall_score','sleep_composition_score',
             'sleep_revitalization_score', 'sleep_duration_score',
             'deep_sleep_in_minutes', 'sleep_resting_heart_rate','sleep_restlessness',
             'sleep_duration','sleep_minutesToFallAsleep','sleep_minutesAsleep','sleep_minutesAwake',
             'sleep_minutesAfterWakeup','sleep_timeInBed','sleep_efficiency',
             'main_sleep','steps',
             'hr_below_zone1', 'hr_zone1', 'hr_zone3','hr_zone2','v_act']

In [13]:
P03

,readiness,calories,distance,l_act,m_act,resting_heart,sedentary,sleep_overall_score,sleep_composition_score,sleep_revitalization_score,...,sleep_minutesAfterWakeup,sleep_timeInBed,sleep_efficiency,main_sleep,steps,hr_below_zone1,hr_zone1,hr_zone3,hr_zone2,v_act
2019-11-01,5.0,3221.06,894500,227,7,60.710199,1035,NaN,NaN,NaN,...,0.0,135.0,93.0,1.0,11204,0.0,81.0,896.0,0.0,36
2019-11-02,NaN,2664.37,354170,91,12,62.794794,888,65.0,17.0,13.0,...,4.0,424.0,89.0,1.0,4435,0.0,54.0,668.0,0.0,25
2019-11-03,5.0,1944.13,0,0,0,0.000000,1440,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
2019-11-04,5.0,1944.00,0,0,0,0.000000,1440,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
2019-11-05,5.0,2706.66,533930,127,11,61.953474,1276,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6710,0.0,59.0,539.0,3.0,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-27,NaN,2771.67,469560,194,11,59.188649,922,NaN,NaN,NaN,...,0.0,172.0,93.0,1.0,5906,0.0,15.0,1187.0,0.0,1
2020-03-28,NaN,1929.60,0,0,0,0.000000,1440,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
2020-03-29,NaN,2101.65,97910,44,0,59.043925,1017,77.0,20.0,21.0,...,0.0,439.0,97.0,1.0,1233,0.0,2.0,983.0,0.0,0
2020-03-30,5.0,2046.35,37730,32,0,57.505357,1044,73.0,18.0,21.0,...,0.0,364.0,91.0,1.0,475,0.0,3.0,601.0,0.0,0
